# 5.1 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0829, -0.0935, -0.2492,  0.0344,  0.0789, -0.1500, -0.0933,  0.1326,
         -0.0192,  0.0936],
        [-0.0561, -0.1301, -0.1633, -0.0379, -0.0156, -0.2141, -0.1026,  0.1895,
         -0.0460, -0.0333]], grad_fn=<AddmmBackward0>)

## 5.1.5 自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.1955, -0.1389, -0.1874, -0.1470, -0.0563,  0.0225, -0.1792, -0.1261,
          0.0284, -0.0372],
        [ 0.1410, -0.1474, -0.0754, -0.2164,  0.0150,  0.0478, -0.1394, -0.1116,
          0.0800, -0.1133]], grad_fn=<AddmmBackward0>)

## 5.1.2 顺序块

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1446, -0.0800,  0.0360, -0.1327,  0.0789, -0.0247, -0.1470,  0.4224,
         -0.2555, -0.2430],
        [ 0.1629, -0.0224, -0.0368, -0.0693,  0.0020, -0.1330, -0.2958,  0.2746,
         -0.1952, -0.1520]], grad_fn=<AddmmBackward0>)

## 5.1.3 在前向传播函数中执行代码

In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        print("执行fix函数")
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        
        return X.sum()

In [11]:
net = FixedHiddenMLP()
net(X)

执行fix函数


tensor(-0.0412, grad_fn=<SumBackward0>)

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

执行fix函数


tensor(-0.1641, grad_fn=<SumBackward0>)

## 5.1.4 效率

# 5.2 参数管理

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[ 0.0489],
        [-0.0440]], grad_fn=<AddmmBackward0>)

## 5.2.1 参数访问


In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2108,  0.3411, -0.1254,  0.2729,  0.0622, -0.0417,  0.3127, -0.1104]])), ('bias', tensor([-0.2269]))])


In [3]:
#目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2269], requires_grad=True)
tensor([-0.2269])


In [4]:
net[2].weight.grad == None

True

In [5]:
#一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
net.state_dict()['2.bias'].data

tensor([-0.2269])

In [7]:
#从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3199],
        [0.3199]], grad_fn=<AddmmBackward0>)

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
rgnet[0][1][0].bias.data

tensor([-0.4228, -0.3167, -0.3014, -0.4914, -0.3234,  0.3718,  0.3965, -0.4383])

## 5.2.2 参数初始化

In [10]:
#内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0093, -0.0083,  0.0047,  0.0205]), tensor(0.))

In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.2830,  0.6223,  0.6168, -0.5090])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [13]:
#自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 9.8583, -0.0000, -5.4187,  0.0000],
        [-0.0000,  6.6931, -8.2820, -6.2823]], grad_fn=<SliceBackward0>)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -4.4187,  1.0000])

## 5.2.3 参数绑定


In [16]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 5.3 延后初始化

# 5.4 自定义层

## 5.4.1 不带参数的层

In [17]:
import torch
import torch.nn.functional as F
from torch import nn

In [19]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [21]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [23]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [24]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-5.1223e-09, grad_fn=<MeanBackward0>)

## 5.4.2 带参数的层

In [27]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [28]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.7462,  0.7987, -1.2150],
        [-0.1240,  0.7548,  0.7987],
        [-0.3407,  0.7146,  0.1463],
        [-0.5147,  0.7992, -0.0102],
        [-0.6151, -2.2530,  0.2700]], requires_grad=True)

In [29]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0375]])

In [30]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.5989],
        [5.5256]])

# 5.5 读写文件

## 5.5.1 加载和保存张量

In [31]:
import torch
from torch import nn 
from torch.nn import functional as F

In [32]:
x = torch.arange(4)
torch.save(x, 'x-file')

In [33]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [34]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [35]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 5.5.2 加载和保存模型参数

In [36]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256 ,10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

In [37]:
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [39]:
torch.save(net.state_dict(), 'mlp.params')

In [40]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [41]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# 5.6 GPU

In [42]:
! nvidia-smi

Mon Feb 27 10:47:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.47       Driver Version: 526.47       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P0    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 5.6.1 计算设备

In [43]:
import torch
from torch import nn
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [44]:
torch.cuda.device_count()

1

In [46]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 5.6.2 张量与GPU

In [47]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [48]:
#存储在GPU上
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [51]:
Y = torch.rand(2, 3, device=try_gpu(1))

In [52]:
Y

tensor([[0.7420, 0.0177, 0.6510],
        [0.5230, 0.9839, 0.3845]])

In [ ]:
#复制
Z = X.cuda(1)
print(X)
print(Z)

## 5.6.3 神经网络与GPU

In [54]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [55]:
net(X)

tensor([[0.0925],
        [0.0925]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [56]:
net[0].weight.data.device

device(type='cuda', index=0)

: 